<a href="https://colab.research.google.com/github/Talismanic/unit-test-using-llms/blob/main/Unit_Test_PoC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# unit_test_poc

# Main Class

In [ ]:
# Main Class from below repository
# https://github.com/realpython/codetiming


"""Dictionary-like structure with information about timers."""

# Standard library imports
import collections
import math
import statistics
from typing import TYPE_CHECKING, Any, Callable, Dict, List

# Annotate generic UserDict
if TYPE_CHECKING:
    UserDict = collections.UserDict[str, float]  # pragma: no cover
else:
    UserDict = collections.UserDict


class Timers(UserDict):
    """Custom dictionary that stores information about timers."""

    def __init__(self, *args: Any, **kwargs: Any) -> None:
        """Add a private dictionary keeping track of all timings"""
        super().__init__(*args, **kwargs)
        self._timings: Dict[str, List[float]] = collections.defaultdict(list)

    def add(self, name: str, value: float) -> None:
        """Add a timing value to the given timer."""
        self._timings[name].append(value)
        self.data.setdefault(name, 0)
        self.data[name] += value

    def clear(self) -> None:
        """Clear timers."""
        self.data.clear()
        self._timings.clear()

    def __setitem__(self, name: str, value: float) -> None:
        """Disallow setting of timer values."""
        raise TypeError(
            f"{self.__class__.__name__!r} does not support item assignment. "
            "Use '.add()' to update values."
        )

    def apply(self, func: Callable[[List[float]], float], name: str) -> float:
        """Apply a function to the results of one named timer."""
        if name in self._timings:
            return func(self._timings[name])
        raise KeyError(name)

    def count(self, name: str) -> float:
        """Number of timings."""
        return self.apply(len, name=name)

    def total(self, name: str) -> float:
        """Total time for timers."""
        return self.apply(sum, name=name)

    def min(self, name: str) -> float:
        """Minimal value of timings."""
        return self.apply(lambda values: min(values or [0]), name=name)

    def max(self, name: str) -> float:
        """Maximal value of timings."""
        return self.apply(lambda values: max(values or [0]), name=name)

    def mean(self, name: str) -> float:
        """Mean value of timings."""
        return self.apply(lambda values: statistics.mean(values or [0]), name=name)

    def median(self, name: str) -> float:
        """Median value of timings."""
        return self.apply(lambda values: statistics.median(values or [0]), name=name)

    def stdev(self, name: str) -> float:
        """Standard deviation of timings."""
        if name in self._timings:
            value = self._timings[name]
            return statistics.stdev(value) if len(value) >= 2 else math.nan
        raise KeyError(name)


# Human Test Code




In [ ]:
"""Tests for codetiming.Timer

Based on the Pytest test runner
"""
# Standard library imports
import math
import re
import time

# Third party imports
import pytest

# Codetiming imports
from codetiming import Timer, TimerError

#
# Constants, functions, and classes used by tests
#
TIME_PREFIX = "Wasted time:"
TIME_MESSAGE = f"{TIME_PREFIX} {{:.4f}} seconds"
RE_TIME_MESSAGE = re.compile(TIME_PREFIX + r" 0\.\d{4} seconds")
RE_TIME_MESSAGE_INITIAL_TEXT_TRUE = re.compile(
    f"Timer started\n{TIME_PREFIX}" + r" 0\.\d{4} seconds"
)
RE_TIME_MESSAGE_INITIAL_TEXT_CUSTOM = re.compile(
    f"Starting the party\n{TIME_PREFIX}" + r" 0\.\d{4} seconds"
)


def waste_time(num: int = 1000) -> None:
    """Just waste a little bit of time."""
    sum(n**2 for n in range(num))


@Timer(text=TIME_MESSAGE)
def decorated_timewaste(num: int = 1000) -> None:
    """Just waste a little bit of time."""
    sum(n**2 for n in range(num))


@Timer(text=TIME_MESSAGE, initial_text=True)
def decorated_timewaste_initial_text_true(num: int = 1000) -> None:
    """Just waste a little bit of time."""
    sum(n**2 for n in range(num))


@Timer(text=TIME_MESSAGE, initial_text="Starting the party")
def decorated_timewaste_initial_text_custom(num: int = 1000) -> None:
    """Just waste a little bit of time."""
    sum(n**2 for n in range(num))


@Timer(name="accumulator", text=TIME_MESSAGE)
def accumulated_timewaste(num: int = 1000) -> None:
    """Just waste a little bit of time."""
    sum(n**2 for n in range(num))


class CustomLogger:
    """Simple class used to test custom logging capabilities in Timer."""

    def __init__(self) -> None:
        """Store log messages in the .messages attribute."""
        self.messages = ""

    def __call__(self, message: str) -> None:
        """Add a log message to the .messages attribute."""
        self.messages += message


#
# Tests
#
def test_timer_as_decorator(capsys: pytest.CaptureFixture[str]) -> None:
    """Test that decorated function prints timing information."""
    decorated_timewaste()

    stdout, stderr = capsys.readouterr()
    assert RE_TIME_MESSAGE.match(stdout)
    assert stdout.count("\n") == 1
    assert stderr == ""


def test_timer_as_context_manager(capsys: pytest.CaptureFixture[str]) -> None:
    """Test that timed context prints timing information."""
    with Timer(text=TIME_MESSAGE):
        waste_time()

    stdout, stderr = capsys.readouterr()
    assert RE_TIME_MESSAGE.match(stdout)
    assert stdout.count("\n") == 1
    assert stderr == ""


def test_explicit_timer(capsys: pytest.CaptureFixture[str]) -> None:
    """Test that timed section prints timing information."""
    t = Timer(text=TIME_MESSAGE)
    t.start()
    waste_time()
    t.stop()

    stdout, stderr = capsys.readouterr()
    assert RE_TIME_MESSAGE.match(stdout)
    assert stdout.count("\n") == 1
    assert stderr == ""


def test_error_if_timer_not_running() -> None:
    """Test that timer raises error if it is stopped before started."""
    t = Timer(text=TIME_MESSAGE)
    with pytest.raises(TimerError):
        t.stop()


def test_access_timer_object_in_context(capsys: pytest.CaptureFixture[str]) -> None:
    """Test that we can access the timer object inside a context."""
    with Timer(text=TIME_MESSAGE) as t:
        assert isinstance(t, Timer)
        assert isinstance(t.text, str)
        assert t.text.startswith(TIME_PREFIX)
    _, _ = capsys.readouterr()  # Do not print log message to standard out


def test_custom_logger() -> None:
    """Test that we can use a custom logger."""
    logger = CustomLogger()
    with Timer(text=TIME_MESSAGE, logger=logger):
        waste_time()
    assert RE_TIME_MESSAGE.match(logger.messages)


def test_timer_without_text(capsys: pytest.CaptureFixture[str]) -> None:
    """Test that timer with logger=None does not print anything."""
    with Timer(logger=None):
        waste_time()

    stdout, stderr = capsys.readouterr()
    assert stdout == ""
    assert stderr == ""


def test_accumulated_decorator(capsys: pytest.CaptureFixture[str]) -> None:
    """Test that decorated timer can accumulate."""
    accumulated_timewaste()
    accumulated_timewaste()

    stdout, stderr = capsys.readouterr()
    lines = stdout.strip().split("\n")
    assert len(lines) == 2
    assert RE_TIME_MESSAGE.match(lines[0])
    assert RE_TIME_MESSAGE.match(lines[1])
    assert stderr == ""


def test_accumulated_context_manager(capsys: pytest.CaptureFixture[str]) -> None:
    """Test that context manager timer can accumulate."""
    t = Timer(name="accumulator", text=TIME_MESSAGE)
    with t:
        waste_time()
    with t:
        waste_time()

    stdout, stderr = capsys.readouterr()
    lines = stdout.strip().split("\n")
    assert len(lines) == 2
    assert RE_TIME_MESSAGE.match(lines[0])
    assert RE_TIME_MESSAGE.match(lines[1])
    assert stderr == ""


def test_accumulated_explicit_timer(capsys: pytest.CaptureFixture[str]) -> None:
    """Test that explicit timer can accumulate."""
    t = Timer(name="accumulated_explicit_timer", text=TIME_MESSAGE)
    total = 0.0
    t.start()
    waste_time()
    total += t.stop()
    t.start()
    waste_time()
    total += t.stop()

    stdout, stderr = capsys.readouterr()
    lines = stdout.strip().split("\n")
    assert len(lines) == 2
    assert RE_TIME_MESSAGE.match(lines[0])
    assert RE_TIME_MESSAGE.match(lines[1])
    assert stderr == ""
    assert total == Timer.timers["accumulated_explicit_timer"]


def test_error_if_restarting_running_timer() -> None:
    """Test that restarting a running timer raises an error."""
    t = Timer(text=TIME_MESSAGE)
    t.start()
    with pytest.raises(TimerError):
        t.start()


def test_last_starts_as_nan() -> None:
    """Test that .last attribute is initialized as nan."""
    t = Timer()
    assert math.isnan(t.last)


def test_timer_sets_last() -> None:
    """Test that .last attribute is properly set."""
    with Timer(logger=None) as t:
        time.sleep(0.02)

    assert t.last >= 0.02


def test_using_name_in_text_without_explicit_timer(
    capsys: pytest.CaptureFixture[str],
) -> None:
    """Test that the name of the timer can be referenced in the text."""
    name = "NamedTimer"
    with Timer(name=name, text="{name}: {:.2f}"):
        waste_time()

    stdout, stderr = capsys.readouterr()
    assert re.match(f"{name}: " + r"0\.\d{2}", stdout)
    assert stderr == ""


def test_using_name_in_text_with_explicit_timer(
    capsys: pytest.CaptureFixture[str],
) -> None:
    """Test that timer name and seconds attribute can be referenced in the text."""
    name = "NamedTimer"
    with Timer(name=name, text="{name}: {seconds:.2f}"):
        waste_time()

    stdout, stderr = capsys.readouterr()
    assert re.match(f"{name}: " + r"0\.\d{2}", stdout.strip())
    assert stderr == ""


def test_using_minutes_attribute_in_text(capsys: pytest.CaptureFixture[str]) -> None:
    """Test that timer can report its duration in minutes."""
    with Timer(text="{minutes:.1f} minutes"):
        waste_time()

    stdout, stderr = capsys.readouterr()
    assert stdout.strip() == "0.0 minutes"
    assert stderr == ""


def test_using_milliseconds_attribute_in_text(
    capsys: pytest.CaptureFixture[str],
) -> None:
    """Test that timer can report its duration in milliseconds."""
    with Timer(text="{milliseconds:.0f} {seconds:.3f}"):
        waste_time()

    stdout, stderr = capsys.readouterr()
    milliseconds, _, seconds = stdout.partition(" ")
    assert int(milliseconds) == round(float(seconds) * 1000)
    assert stderr == ""


def test_text_formatting_function(capsys: pytest.CaptureFixture[str]) -> None:
    """Test that text can be formatted by a separate function."""

    def format_text(seconds: float) -> str:
        """Function that returns a formatted text"""
        return f"Function: {seconds + 1:.0f}"

    with Timer(text=format_text):
        waste_time()

    stdout, stderr = capsys.readouterr()
    assert stdout.strip() == "Function: 1"
    assert not stderr.strip()


def test_text_formatting_class(capsys: pytest.CaptureFixture[str]) -> None:
    """Test that text can be formatted by a separate class."""

    class TextFormatter:
        """Class that behaves like a formatted text."""

        def __init__(self, seconds: float) -> None:
            """Store the elapsed number of seconds"""
            self.seconds = seconds

        def __str__(self) -> str:
            """Represent the class as a formatted text"""
            return f"Class: {self.seconds + 1:.0f}"

    with Timer(text=TextFormatter):
        waste_time()

    stdout, stderr = capsys.readouterr()
    assert stdout.strip() == "Class: 1"
    assert not stderr.strip()


def test_timers_cleared() -> None:
    """Test that timers can be cleared."""
    with Timer(name="timer_to_be_cleared", logger=None):
        waste_time()

    assert "timer_to_be_cleared" in Timer.timers
    Timer.timers.clear()
    assert not Timer.timers


def test_running_cleared_timers(capsys: pytest.CaptureFixture[str]) -> None:
    """Test that timers can still be run after they're cleared."""
    t = Timer(name="timer_to_be_cleared")
    Timer.timers.clear()

    accumulated_timewaste()
    with t:
        waste_time()

    capsys.readouterr()
    assert "accumulator" in Timer.timers
    assert "timer_to_be_cleared" in Timer.timers


def test_timers_stats() -> None:
    """Test that we can get basic statistics from timers."""
    name = "timer_with_stats"
    t = Timer(name=name, logger=None)
    for num in range(5, 10):
        with t:
            waste_time(num=100 * num)

    stats = Timer.timers
    assert stats.total(name) == stats[name]
    assert stats.count(name) == 5
    assert stats.min(name) <= stats.median(name) <= stats.max(name)
    assert stats.mean(name) >= stats.min(name)
    assert stats.stdev(name) >= 0


def test_stats_missing_timers() -> None:
    """Test that getting statistics from non-existent timers raises exception."""
    with pytest.raises(KeyError):
        Timer.timers.count("non_existent_timer")

    with pytest.raises(KeyError):
        Timer.timers.stdev("non_existent_timer")


def test_setting_timers_exception() -> None:
    """Test that setting .timers items raises exception."""
    with pytest.raises(TypeError):
        Timer.timers["set_timer"] = 1.23


def test_timer_as_decorator_with_initial_text_true(
    capsys: pytest.CaptureFixture[str],
) -> None:
    """Test that decorated function prints at start with default initial text."""
    decorated_timewaste_initial_text_true()

    stdout, stderr = capsys.readouterr()
    assert RE_TIME_MESSAGE_INITIAL_TEXT_TRUE.match(stdout)
    assert stdout.count("\n") == 2
    assert stderr == ""


def test_timer_as_context_manager_with_initial_text_true(
    capsys: pytest.CaptureFixture[str],
) -> None:
    """Test that timed context prints at start with default initial text."""
    with Timer(text=TIME_MESSAGE, initial_text=True):
        waste_time()

    stdout, stderr = capsys.readouterr()
    assert RE_TIME_MESSAGE_INITIAL_TEXT_TRUE.match(stdout)
    assert stdout.count("\n") == 2
    assert stderr == ""


def test_explicit_timer_with_initial_text_true(
    capsys: pytest.CaptureFixture[str],
) -> None:
    """Test that timed section prints at start with default initial text."""
    t = Timer(text=TIME_MESSAGE, initial_text=True)
    t.start()
    waste_time()
    t.stop()

    stdout, stderr = capsys.readouterr()
    assert RE_TIME_MESSAGE_INITIAL_TEXT_TRUE.match(stdout)
    assert stdout.count("\n") == 2
    assert stderr == ""


def test_timer_as_decorator_with_initial_text_custom(
    capsys: pytest.CaptureFixture[str],
) -> None:
    """Test that decorated function prints at start with custom initial text."""
    decorated_timewaste_initial_text_custom()

    stdout, stderr = capsys.readouterr()
    assert RE_TIME_MESSAGE_INITIAL_TEXT_CUSTOM.match(stdout)
    assert stdout.count("\n") == 2
    assert stderr == ""


def test_timer_as_context_manager_with_initial_text_custom(
    capsys: pytest.CaptureFixture[str],
) -> None:
    """Test that timed context prints at start with custom initial text."""
    with Timer(text=TIME_MESSAGE, initial_text="Starting the party"):
        waste_time()

    stdout, stderr = capsys.readouterr()
    assert RE_TIME_MESSAGE_INITIAL_TEXT_CUSTOM.match(stdout)
    assert stdout.count("\n") == 2
    assert stderr == ""


def test_explicit_timer_with_initial_text_custom(
    capsys: pytest.CaptureFixture[str],
) -> None:
    """Test that timed section prints at start with custom initial text."""
    t = Timer(text=TIME_MESSAGE, initial_text="Starting the party")
    t.start()
    waste_time()
    t.stop()

    stdout, stderr = capsys.readouterr()
    assert RE_TIME_MESSAGE_INITIAL_TEXT_CUSTOM.match(stdout)
    assert stdout.count("\n") == 2
    assert stderr == ""


def test_explicit_timer_with_initial_text_true_with_name(
    capsys: pytest.CaptureFixture[str],
) -> None:
    """Test with default initial text referencing timer name."""
    t = Timer(name="named", text=TIME_MESSAGE, initial_text=True)
    t.start()
    waste_time()
    t.stop()

    stdout, stderr = capsys.readouterr()
    assert re.match(
        f"Timer named started\n{TIME_PREFIX}" + r" 0\.\d{4} seconds", stdout
    )
    assert stdout.count("\n") == 2
    assert stderr == ""


def test_explicit_timer_with_initial_text_with_name(
    capsys: pytest.CaptureFixture[str],
) -> None:
    """Test with custom initial text referencing timer name."""
    t = Timer(name="the party", text=TIME_MESSAGE, initial_text="Starting {name}")
    t.start()
    waste_time()
    t.stop()

    stdout, stderr = capsys.readouterr()
    assert RE_TIME_MESSAGE_INITIAL_TEXT_CUSTOM.match(stdout)
    assert stdout.count("\n") == 2
    assert stderr == ""

# Pynguin Test Code

In [ ]:
# Test cases automatically generated by Pynguin (https://www.pynguin.eu).
# Please check them before you use them.
import pytest
import _timers as module_0


def test_case_0():
    timers_0 = module_0.Timers()
    assert len(timers_0) == 0


@pytest.mark.xfail(strict=True)
def test_case_1():
    dict_0 = {}
    timers_0 = module_0.Timers(**dict_0)
    assert len(timers_0) == 0
    str_0 = "h$cTPYHD?.UwWB:G[\\4h"
    timers_0.min(str_0)


def test_case_2():
    int_0 = 398
    timers_0 = module_0.Timers()
    assert len(timers_0) == 0
    str_0 = "\x0bt9gFnz#'T l5r*"
    none_type_0 = timers_0.add(str_0, int_0)
    assert len(timers_0) == 1
    float_0 = timers_0.min(str_0)
    assert float_0 == 398


@pytest.mark.xfail(strict=True)
def test_case_3():
    timers_0 = module_0.Timers()
    assert len(timers_0) == 0
    none_type_0 = timers_0.clear()
    timers_0.min(none_type_0)


@pytest.mark.xfail(strict=True)
def test_case_4():
    timers_0 = module_0.Timers()
    assert len(timers_0) == 0
    timers_0.__setitem__(timers_0, timers_0)


@pytest.mark.xfail(strict=True)
def test_case_5():
    dict_0 = {}
    timers_0 = module_0.Timers(**dict_0)
    assert len(timers_0) == 0
    str_0 = ""
    timers_0.count(str_0)


@pytest.mark.xfail(strict=True)
def test_case_6():
    int_0 = 398
    timers_0 = module_0.Timers()
    assert len(timers_0) == 0
    timers_0.total(int_0)


@pytest.mark.xfail(strict=True)
def test_case_7():
    timers_0 = module_0.Timers()
    assert len(timers_0) == 0
    var_0 = timers_0.__copy__()
    assert f"{type(var_0).__module__}.{type(var_0).__qualname__}" == "_timers.Timers"
    assert len(var_0) == 0
    var_1 = var_0.__ior__(var_0)
    timers_0.max(var_1)


def test_case_8():
    int_0 = 380
    timers_0 = module_0.Timers()
    assert len(timers_0) == 0
    str_0 = "^N,4|1|"
    none_type_0 = timers_0.add(str_0, int_0)
    assert len(timers_0) == 1
    float_0 = timers_0.mean(str_0)
    assert float_0 == 380


def test_case_9():
    int_0 = 398
    timers_0 = module_0.Timers()
    assert len(timers_0) == 0
    str_0 = "^N,4\r1|"
    none_type_0 = timers_0.add(str_0, int_0)
    assert len(timers_0) == 1
    float_0 = timers_0.median(str_0)
    assert float_0 == 398


def test_case_10():
    timers_0 = module_0.Timers()
    assert len(timers_0) == 0
    str_0 = "^N,4|1|"
    with pytest.raises(KeyError):
        timers_0.stdev(str_0)


@pytest.mark.xfail(strict=True)
def test_case_11():
    int_0 = 398
    timers_0 = module_0.Timers()
    assert len(timers_0) == 0
    str_0 = "^N,4|1|"
    none_type_0 = timers_0.add(str_0, int_0)
    assert len(timers_0) == 1
    float_0 = timers_0.stdev(str_0)
    timers_0.count(int_0)


def test_case_12():
    int_0 = 380
    timers_0 = module_0.Timers()
    assert len(timers_0) == 0
    str_0 = "^N,4|1|"
    none_type_0 = timers_0.add(str_0, int_0)
    assert len(timers_0) == 1
    float_0 = timers_0.max(str_0)
    assert float_0 == 380


@pytest.mark.xfail(strict=True)
def test_case_13():
    int_0 = 398
    timers_0 = module_0.Timers()
    assert len(timers_0) == 0
    str_0 = "^N,4\r1|"
    bool_0 = False
    none_type_0 = timers_0.add(str_0, bool_0)
    assert len(timers_0) == 1
    none_type_1 = timers_0.add(str_0, int_0)
    float_0 = timers_0.mean(str_0)
    assert float_0 == 199
    float_1 = timers_0.stdev(str_0)
    assert float_1 == pytest.approx(281.4284989122459, abs=0.01, rel=0.01)
    float_2 = timers_0.median(str_0)
    assert float_2 == pytest.approx(199.0, abs=0.01, rel=0.01)
    float_3 = timers_0.min(str_0)
    assert float_3 is False
    float_4 = timers_0.max(str_0)
    assert float_4 == 398
    str_1 = "m8.q&]QoZSPSBpdE"
    timers_0.median(str_1)

# Bard Generated Test Code

In [ ]:
import pytest

from timers import Timers


@pytest.fixture
def timers() -> Timers:
    """Create a new `Timers` instance."""
    return Timers()


def test_add(timers: Timers) -> None:
    """Test that the `add()` method adds a timing value to the given timer."""
    timers.add("foo", 1.0)
    assert timers["foo"] == [1.0]


def test_clear(timers: Timers) -> None:
    """Test that the `clear()` method clears all timers."""
    timers.add("foo", 1.0)
    timers.clear()
    assert timers == {}


def test_apply(timers: Timers) -> None:
    """Test that the `apply()` method applies a function to the results of one named timer."""
    timers.add("foo", 1.0)
    timers.add("bar", 2.0)
    assert timers.apply(len, "foo") == 1
    assert timers.apply(len, "bar") == 1
    assert timers.apply(sum, "foo") == 1.0
    assert timers.apply(sum, "bar") == 2.0


def test_count(timers: Timers) -> None:
    """Test that the `count()` method returns the number of timings."""
    timers.add("foo", 1.0)
    timers.add("bar", 2.0)
    assert timers.count("foo") == 1
    assert timers.count("bar") == 1


def test_total(timers: Timers) -> None:
    """Test that the `total()` method returns the total time for timers."""
    timers.add("foo", 1.0)
    timers.add("bar", 2.0)
    assert timers.total("foo") == 1.0
    assert timers.total("bar") == 2.0


def test_min(timers: Timers) -> None:
    """Test that the `min()` method returns the minimal value of timings."""
    timers.add("foo", 1.0)
    timers.add("bar", 2.0)
    assert timers.min("foo") == 1.0
    assert timers.min("bar") == 2.0


def test_max(timers: Timers) -> None:
    """Test that the `max()` method returns the maximal value of timings."""
    timers.add("foo", 1.0)
    timers.add("bar", 2.0)
    assert timers.max("foo") == 1.0
    assert timers.max("bar") == 2.0


def test_mean(timers: Timers) -> None:
    """Test that the `mean()` method returns the mean value of timings."""
    timers.add("foo", 1.0)
    timers.add("bar", 2.0)
    assert timers.mean("foo") == 1.5
    assert timers.mean("bar") == 1.5


def test_median(timers: Timers) -> None:
    """Test that the `median()` method returns the median value of timings."""
    timers.add("foo", 1.0)
    timers.add("bar", 2.0)
    assert timers.median("foo") == 1.5
    assert timers.median("bar") == 1.5


def test_stdev(timers: Timers) -> None:
    """Test that the `stdev()` method returns the standard deviation of timings."""
    timers.add("foo", 1.0)
    timers.add("bar", 2.0)
    assert timers.stdev("foo") == 1.0
    assert timers.stdev("bar") == 1.0


if __name__ == "__main__":
    pytest.main()

# Chatgpt Test Code

In [ ]:
import pytest

from my_module import Timers


@pytest.fixture
def timers():
    return Timers()


def test_add(timers):
    timers.add("timer1", 10.0)
    assert timers.data == {"timer1": 10.0}

    timers.add("timer1", 5.0)
    assert timers.data == {"timer1": 15.0}

    timers.add("timer2", 7.5)
    assert timers.data == {"timer1": 15.0, "timer2": 7.5}


def test_clear(timers):
    timers.add("timer1", 10.0)
    timers.add("timer2", 5.0)
    timers.clear()
    assert timers.data == {}
    assert timers._timings == {}


def test_apply(timers):
    timers.add("timer1", 10.0)
    timers.add("timer1", 20.0)
    assert timers.apply(len, name="timer1") == 2
    assert timers.apply(sum, name="timer1") == 30.0
    with pytest.raises(KeyError):
        timers.apply(sum, name="timer2")


def test_count(timers):
    timers.add("timer1", 10.0)
    timers.add("timer1", 20.0)
    assert timers.count("timer1") == 2
    with pytest.raises(KeyError):
        timers.count("timer2")


def test_total(timers):
    timers.add("timer1", 10.0)
    timers.add("timer1", 20.0)
    assert timers.total("timer1") == 30.0
    with pytest.raises(KeyError):
        timers.total("timer2")


def test_min(timers):
    timers.add("timer1", 10.0)
    timers.add("timer1", 5.0)
    assert timers.min("timer1") == 5.0
    assert timers.min("timer2") == 0.0


def test_max(timers):
    timers.add("timer1", 10.0)
    timers.add("timer1", 20.0)
    assert timers.max("timer1") == 20.0
    assert timers.max("timer2") == 0.0


def test_mean(timers):
    timers.add("timer1", 10.0)
    timers.add("timer1", 20.0)
    assert timers.mean("timer1") == 15.0
    assert timers.mean("timer2") == 0.0


def test_median(timers):
    timers.add("timer1", 10.0)
    timers.add("timer1", 20.0)
    assert timers.median("timer1") == 15.0
    assert timers.median("timer2") == 0.0


def test_stdev(timers):
    timers.add("timer1", 10.0)
    timers.add("timer1", 20.0)
    assert timers.stdev("timer1") == pytest.approx(7.0710678118654755)
    with pytest.raises(KeyError):
        timers.stdev("timer2")
